## Creating an AzureML Environment for Stan
First, we'll create a Docker container by way of an AzureML Environment. 

For this, I've written a [Dockerfile](../Dockerfile) that downloads and installs Stan. This is based on the IntelMPI image published by the Azure ML team. (You can find the Dockerfile for that image [here](https://github.com/Azure/AzureML-Containers/blob/master/base/cpu/intelmpi2018.3-ubuntu16.04/Dockerfile).)

In [4]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()

In [11]:
# Read our Dockerfile into a Python string object. 
with open('../Dockerfile', 'r') as file_obj:
    docker_file = file_obj.read()

In [24]:
# Instatiate an Environment object from AML. We've named it 'stan-intelmpi'
env = Environment('stan-intelmpi')

# Next, set the Docker settings for the environment - first setting Docker to enabled, then setting the 
# base_dockerfile property to the Docker string we published before.
# Note - we set the base_image property to None - as base_image and base_dockerfile are mutually exclusive
env.docker.enabled = True
env.docker.base_image = None
env.docker.base_dockerfile = docker_file

# Next, set the Python settings. This will set the pip or conda packages that need to be installed in the container.
env.python.conda_dependencies.set_python_version('3.7.6')
env.python.conda_dependencies.add_pip_package('azureml-defaults')
env.python.conda_dependencies.add_pip_package('cmdstanpy==0.8.0')
env.python.conda_dependencies.add_pip_package('mpi4py==3.0.3')
env.python.conda_dependencies.add_pip_package('azureml-dataprep[fuse,pandas]')

# Finally, register the environment.
env = env.register(ws)

In [26]:
# This is NOT required, but you can prebuild the Docker container by running the env.build step.
# If you choose not to do this, the container will be build the first time you submit the model for building

env_build = env.build(ws)
env_build.wait_for_completion(show_output=True)

In [28]:
env_build.wait_for_completion(show_output=True)

Image Build Status: Running

2020/01/24 19:35:41 Downloading source code...
2020/01/24 19:35:43 Finished downloading source code
2020/01/24 19:35:44 Creating Docker network: acb_default_network, driver: 'bridge'
2020/01/24 19:35:44 Successfully set up Docker network: acb_default_network
2020/01/24 19:35:44 Setting up Docker configuration...
2020/01/24 19:35:45 Successfully set up Docker configuration
2020/01/24 19:35:45 Logging in to registry: amldemocr.azurecr.io
2020/01/24 19:35:48 Successfully logged into amldemocr.azurecr.io
2020/01/24 19:35:48 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/01/24 19:35:48 Scanning for dependencies...
2020/01/24 19:35:49 Successfully scanned dependencies
2020/01/24 19:35:49 Launching container with name: acb_step_0
Sending build context to Docker daemon  61.95kB


Step 1/19 : FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04
intelmpi2018.3-ubuntu16.04: Pulling from azureml/b